**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # use CPU

import time
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The learned action is easily to get stuck at local minimum, so we need adopt some mechanism such that the learned action can escape from it. In order to achieve this, we give a chance for the agent to randomly explore the environment with probability $\epsilon$. And the probability of $1-\epsilon$ to exploit; that is using the action that has already been learned.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.position[self.x, self.y] = 1
        if action == 0: # left
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # right
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # down
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # up
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=301 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The var position is used to record the position of the rat. And the var board is used to indicate the location of cheese and poison at current time state. And from var board, we can visualize the image of current state. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        '''
        Random action taken by RandomAgent.

        Args:
            s: the current state matrix
        Return:
            action taken. Including [0, 1, 2, 3] which correspond to [left, right, down, up]
        '''
        return np.random.randint(4, size=1)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            action = agent.learned_act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            
        # Save as a mp4
        env.draw('Videos/'+prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('Videos/random0.mp4'))

Win/lose count 11.5/11.0. Average score (0.5)
Win/lose count 12.5/18.0. Average score (-2.5)
Win/lose count 6.0/10.0. Average score (-3.0)
Win/lose count 8.0/12.0. Average score (-3.25)
Win/lose count 13.5/17.0. Average score (-3.3)
Win/lose count 8.5/12.0. Average score (-3.3333333333333335)
Win/lose count 12.0/14.0. Average score (-3.142857142857143)
Win/lose count 6.5/8.0. Average score (-2.9375)
Win/lose count 10.0/13.0. Average score (-2.9444444444444446)
Win/lose count 9.0/14.0. Average score (-3.15)
Win/lose count 13.0/13.0. Average score (-2.8636363636363638)
Final score: -2.8636363636363638


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Show that $Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$**

$$\displaystyle \begin{aligned}
Q^\pi (s, a) &= \mathbb{E}\bigg[ \sum_{t=0}^\infty \gamma^t R_{t+1} | S_0=s_0, A_0=a_0 \bigg] \\
&=\sum_{s_1} p(s_1|s_0, a_0) R_1(s_0, a_0, s_1) + \gamma \sum_{s_1, a_1, s_2} p(s_1|s_0, a_0)\pi(a_1|s_1)p(s_2|s_1, a_1)R_2(s_1, a_1, s_2) + \gamma^2 \sum_{s_1, a_1, s_2, a_2, s_3}\dots \\
&= \mathbb{E}_{(s',a')\sim p(.|s,a)}[R(s,a,s')+\gamma Q^{\pi}(s',a')]
\end{aligned}$$

**Show that $Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$**

$$\displaystyle \begin{aligned}
Q^* (s, a) &= \mathbb{E}\bigg[ \sum_{t=0}^\infty \gamma^t R_{t+1} | S_0=s_0, A_0=a_0 \bigg] \\
&=\sum_{s_1} p(s_1|s_0, a_0) R_1(s_0, a_0, s_1) + \gamma \sum_{s_1, a_1, s_2} p(s_1|s_0, a_0)\pi^*(a_1|s_1)p(s_2|s_1, a_1)R_2(s_1, a_1, s_2) + \gamma^2 \sum_{s_1, a_1, s_2, a_2, s_3}\dots \\
&= \mathbb{E}_{(s',a')\sim p(.|s,a)}[R(s,a,s')+\max_{a'}\gamma Q^*(s',a')]
\end{aligned}$$

The second and third equation is equal since $Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \>$.

**Show that $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$**

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory += [m]
        else:
            self.memory = self.memory[1:]
            self.memory += [m]
            
    def random_access(self, batch_size):
        if len(self.memory)<batch_size:
            return random.sample(self.memory, len(self.memory))
        else:
            return random.sample(self.memory, batch_size)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        loss = 0
        win = 0
        lose = 0
        start = time.time()
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state,  action, reward, game_over)
        
        end = time.time()
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw('Videos/'+prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Time {}"
              .format(e, epoch, loss, win, lose, win-lose, end-start))
        agent.save(name_weights='Models/'+prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers import Flatten
from keras.layers import Dropout
import pdb
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        reward_array = self.model.predict(np.expand_dims(s, axis=0))
        action = np.argmax(reward_array)
        return action

    def reinforce(self, prev_state, state,  action, reward, game_over):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([prev_state, state, action, reward, game_over])
        sample_memory = self.memory.random_access(self.batch_size)
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for idx, sample in enumerate(sample_memory):
            _prev_state, _state, _action, _reward, _game_over = sample
            
            input_states[idx] = _prev_state
            _target_q = self.model.predict(np.expand_dims(_prev_state, axis=0))
            
            if _game_over:
                _target_q[0, _action] = _reward
                target_q[idx] = _target_q
            else:
                _next_reward_array = self.model.predict(np.expand_dims(_state, axis=0))
                _next_reward = np.max(_next_reward_array)
                _target_q[0, _action] = _reward + self.discount*_next_reward
                target_q[idx] = _target_q
        
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        
        return l
        
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
                Flatten(), 
                #Dense(300, input_dim=5*5*2, activation='relu'),
                #Dropout(0.2),
                #Dense(200, activation='relu'), 
                #Dropout(0.2),
                Dense(100, activation='relu'), 
                #Dropout(0.2),
                #Dense(50, activation='relu'),
                #Dropout(0.2),
                Dense(4),
        ])
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(adam(lr=lr), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.005, epsilon=0.1, memory_size=100, batch_size=32) # best
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('Videos/fc_train100.mp4'))

Epoch 000/301 | Loss 2.0854 | Win/lose count 3.0/3.0 (0.0) | Time 5.397021532058716
Epoch 001/301 | Loss 2.2446 | Win/lose count 3.0/1.0 (2.0) | Time 3.4677352905273438
Epoch 002/301 | Loss 2.1959 | Win/lose count 3.5/4.0 (-0.5) | Time 3.558528184890747
Epoch 003/301 | Loss 3.7676 | Win/lose count 4.5/7.0 (-2.5) | Time 3.388746500015259
Epoch 004/301 | Loss 2.2804 | Win/lose count 0.5/1.0 (-0.5) | Time 3.434058904647827
Epoch 005/301 | Loss 2.3463 | Win/lose count 2.5/3.0 (-0.5) | Time 3.398329019546509
Epoch 006/301 | Loss 1.9687 | Win/lose count 3.0/5.0 (-2.0) | Time 3.3401448726654053
Epoch 007/301 | Loss 2.1889 | Win/lose count 4.0/5.0 (-1.0) | Time 3.4142210483551025
Epoch 008/301 | Loss 3.0425 | Win/lose count 5.5/5.0 (0.5) | Time 3.402371644973755
Epoch 009/301 | Loss 1.9931 | Win/lose count 6.0/8.0 (-2.0) | Time 3.4545254707336426
Epoch 010/301 | Loss 1.9634 | Win/lose count 8.0/5.0 (3.0) | Time 3.3613245487213135
Epoch 011/301 | Loss 2.5931 | Win/lose count 8.5/4.0 (4.5) | Tim

In [13]:
test(agent,env,epochs_test,prefix='fc_test')
HTML(display_videos('Videos/fc_test30.mp4'))

Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.25)
Win/lose count 0.5/0. Average score (0.3333333333333333)
Win/lose count 3.0/0. Average score (1.0)
Win/lose count 0/0. Average score (0.8)
Win/lose count 2.5/0. Average score (1.0833333333333333)
Win/lose count 0/0. Average score (0.9285714285714286)
Win/lose count 0.5/0. Average score (0.875)
Win/lose count 4.0/1.0. Average score (1.1111111111111112)
Win/lose count 1.0/0. Average score (1.1)
Win/lose count 3.5/0. Average score (1.3181818181818181)
Final score: 1.3181818181818181


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
                Conv2D(filters=16,
                       kernel_size=(3, 3),  
                       padding='same',
                       input_shape=(5, 5, self.n_state),
                       activation='relu'),
                Flatten(),
                Dense(100, activation='relu'), 
                Dense(4),
        ])
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(adam(lr=lr), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=100, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('videos/cnn_train10.mp4'))

Epoch 000/301 | Loss 1.1484 | Win/lose count 1.5/3.0 (-1.5) | Time 4.442844867706299
Epoch 001/301 | Loss 2.3100 | Win/lose count 2.5/4.0 (-1.5) | Time 4.098010540008545
Epoch 002/301 | Loss 2.9493 | Win/lose count 5.0/2.0 (3.0) | Time 4.1389007568359375
Epoch 003/301 | Loss 2.4908 | Win/lose count 8.5/8.0 (0.5) | Time 4.095983982086182
Epoch 004/301 | Loss 1.5335 | Win/lose count 6.0/7.0 (-1.0) | Time 4.100005388259888
Epoch 005/301 | Loss 2.1091 | Win/lose count 5.0/7.0 (-2.0) | Time 4.0960142612457275
Epoch 006/301 | Loss 1.6447 | Win/lose count 10.5/4.0 (6.5) | Time 4.087040662765503
Epoch 007/301 | Loss 2.5157 | Win/lose count 8.5/5.0 (3.5) | Time 4.10798454284668
Epoch 008/301 | Loss 2.3851 | Win/lose count 12.5/6.0 (6.5) | Time 4.100006580352783
Epoch 009/301 | Loss 1.6989 | Win/lose count 9.5/3.0 (6.5) | Time 4.114965915679932
Epoch 010/301 | Loss 1.4103 | Win/lose count 4.5/1.0 (3.5) | Time 4.095019340515137
Epoch 011/301 | Loss 1.6491 | Win/lose count 7.5/3.0 (4.5) | Time 4.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='Models/cnn_trainmodel.h5', name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='Models/fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 1.0/0. Average score (2.0)
Win/lose count 0.5/0. Average score (1.5)
Win/lose count 0/0. Average score (1.125)
Win/lose count 0/0. Average score (0.9)
Win/lose count 2.0/0. Average score (1.0833333333333333)
Win/lose count 1.0/0. Average score (1.0714285714285714)
Win/lose count 1.5/0. Average score (1.125)
Win/lose count 3.0/0. Average score (1.3333333333333333)
Win/lose count 0/0. Average score (1.2)
Win/lose count 1.0/0. Average score (1.1818181818181819)
Final score: 1.1818181818181819
Test of the FC
Win/lose count 1.0/4.0. Average score (-3.0)
Win/lose count 1.5/2.0. Average score (-1.75)
Win/lose count 1.0/0. Average score (-0.8333333333333334)
Win/lose count 1.5/3.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 1.0/2.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lo

In [17]:
HTML(display_videos('Videos/cnn_test10.mp4'))

In [18]:
HTML(display_videos('Videos/fc_test10.mp4'))

I ran several tests on cnn and dnn. Overall, CNN got better score than DNN since it consider the spatial information of the board. It is easier for the model to learn the $Q$ function. In addition, we found that both algorithm always stuck in some states and could not escape. The reason is that while training, the agent did not explore the environment but tend to take some certain actions.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent, env, epoch, eps_decay=0.999, eps_lower=0.1, prefix=''):
    score = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        loss = 0
        win = 0
        lose = 0
        start = time.time()
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            epsilon = agent.epsilon
            if epsilon > eps_lower:
                epsilon *= eps_decay
            else:
                epsilon = eps_lower
            agent.set_epsilon(epsilon)
            loss += agent.reinforce(prev_state, state,  action, reward, game_over)
        
        end = time.time()
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw('Videos/'+prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.4f}/{:.4f} ({:.4f}) | Time {}"
              .format(e, epoch, loss, win, lose, win-lose, end-start))
        agent.save(name_weights='Models/'+prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
    
    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.position[self.x, self.y] = 1
        if action == 0: # left
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # right
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # down
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # up
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.005, epsilon=0.3, memory_size=500, batch_size=50, n_state=3)
train_explore(agent, env, epochs_train, eps_decay=1.0, prefix='cnn_train_explore')
HTML(display_videos('Videos/cnn_train_explore10.mp4'))

Epoch 000/301 | Loss 2.0996 | Win/lose count 4.5000/23.3000 (-18.8000) | Time 7.504314661026001
Epoch 001/301 | Loss 2.4758 | Win/lose count 17.4000/23.9000 (-6.5000) | Time 6.6122355461120605
Epoch 002/301 | Loss 1.6344 | Win/lose count 13.9000/20.4000 (-6.5000) | Time 6.649142503738403
Epoch 003/301 | Loss 1.8624 | Win/lose count 11.5000/26.2000 (-14.7000) | Time 6.6212451457977295
Epoch 004/301 | Loss 1.9844 | Win/lose count 15.1000/22.2000 (-7.1000) | Time 6.65313196182251
Epoch 005/301 | Loss 1.2572 | Win/lose count 15.5000/28.8000 (-13.3000) | Time 6.615229845046997
Epoch 006/301 | Loss 1.1522 | Win/lose count 11.3000/22.3000 (-11.0000) | Time 6.69404935836792
Epoch 007/301 | Loss 1.1445 | Win/lose count 11.2000/28.2000 (-17.0000) | Time 6.7239696979522705
Epoch 008/301 | Loss 0.8769 | Win/lose count 11.6000/22.9000 (-11.3000) | Time 6.629223346710205
Epoch 009/301 | Loss 1.2009 | Win/lose count 12.8000/27.9000 (-15.1000) | Time 6.619248867034912
Epoch 010/301 | Loss 1.2727 | Win

In [21]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('Videos/cnn_test_explore10.mp4'))

Win/lose count 0.8/19.900000000000013. Average score (-19.100000000000012)
Win/lose count 3.1999999999999997/19.300000000000004. Average score (-17.60000000000001)
Win/lose count 2.4/20.50000000000002. Average score (-17.76666666666668)
Win/lose count 2.0/19.60000000000001. Average score (-17.725000000000012)
Win/lose count 1.2000000000000002/19.80000000000001. Average score (-17.900000000000013)
Win/lose count 3.1999999999999997/19.300000000000004. Average score (-17.60000000000001)
Win/lose count 1.2000000000000002/20.80000000000003. Average score (-17.885714285714297)
Win/lose count 0.4/21.000000000000032. Average score (-18.225000000000012)
Win/lose count 0.4/20.000000000000014. Average score (-18.377777777777787)
Win/lose count 2.0/19.60000000000001. Average score (-18.30000000000001)
Win/lose count 1.6/20.700000000000028. Average score (-18.372727272727285)
Final score: -18.372727272727285


In [22]:
HTML(display_videos('Videos/cnn_test_explore5.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***